# Домашнее задание 2.2. Обучение сетей на Pytorch

В этом задании нужно:
1. Написать свою сеть на Pytorch по варианту
2. Обучить ее и сравнить результаты с дообученной сетью из зоопарка моделей
3. Поставить ряд экспериментов, показывающих насколько гиперпараметры обучения влияют на результат

**Варианты архитектуры сверточной сети:**
Вариант на ваш выбор - напишите его в чат. Не более двух человек на один вариант
1. Resnet v2
2. Inception Google LeNet
3. MobileNetv2 (Коростинский, Глаз)
4. SE Net
5. DenseNet
6. Conv Mixer
7. RepVGG

## Имплементация сети на Pytorch

Здесь вы должны написать модель, выданную вам по варианту.
Для этого нужно:
1. Не забывать про использоватие блоков nn.Module, nn.Sequential, nn.ModuleList
2. Использовать материалы из предыдущих семинаров

В качестве примера ниже реализован макет для модели, состоящей из блоков.

## Архитектура Dense слоя сети DenseNet. 
В нем все  признаки с каждого слоя конкатенируются и передаются во все следующие.

![image.png](DenseNet.jpg)

In [122]:
import torch
from torch import nn
from collections import OrderedDict

In [123]:
# Input: (N, C, H, W) tensor
# Layer: combination of (BN2D + ReLu + Conv2D1x1 + BN2D + ReLu + Conv2D3x3) layers
# Arch: each layer l accepts k_0 + k * (l - 1) feature maps
# There is a bottleneck in each layer: each Conv2D1x1 reduces number of chanels to bn_size
# This is needed because first convolution might have high num_input_features 
#   because of data from the previous layers:
# Example of how C changes throughout the network:
# 	layer 1: (input) -> (bn_size * k) -> (k) 
#		 		|						 |
# 		 		|						 V
# 	layer 2: 	--------------> (input + k) -> (bn_size * k) -> (k)
#   layer 3:    (input + k + k) -> (bn_size * k) -> (k)

class DenseLayer(nn.Module):
	def __init__(self, num_input_features : int, bottle_neck_dim : int, num_out_features : int) -> None:
		super().__init__()
		self.bottle_neck = nn.Sequential(OrderedDict([ 
			('norm1', nn.BatchNorm2d(num_input_features)),
			('relu1', nn.ReLU()),
			('conv1', nn.Conv2d(num_input_features, bottle_neck_dim,
							kernel_size=1, stride=1, bias=False))]))
		
		self.conv = nn.Sequential(OrderedDict([
			('norm2', nn.BatchNorm2d(bottle_neck_dim)),
			('relu2', nn.ReLU()),
			# Pudding is needed in order to match concatinating outputs
			('conv2', nn.Conv2d(bottle_neck_dim, num_out_features,
							kernel_size=3, stride=1, padding=1, bias=False)),
		]))
		
	def forward(self, input : torch.tensor) -> torch.tensor:
		bottle_neck_output = self.bottle_neck(input)
		return self.conv(bottle_neck_output)

class DenseBlock(nn.Module):
	def __init__(self, num_input_features : int, num_layers : int, k : int, bn_size : int) -> None:
		super().__init__()
		self.dense_blocks = nn.Sequential()
		cur_num_input_features = num_input_features
		for l in range(num_layers):
			layer = DenseLayer(num_input_features=cur_num_input_features, 
							   bottle_neck_dim=bn_size, 
							   num_out_features=k)
			self.dense_blocks.add_module(f"denseblock{l}", layer)
			cur_num_input_features = (l + 1) * k
		
	def forward(self, input : torch.tensor) -> torch.tensor:
		prev_features = None
		for layer in self.dense_blocks:
			output = layer(input)
			# dim=1 is a dimension of blocks
			if prev_features is not None:
				input = torch.cat((prev_features, output), dim=1)
			else:
				input = output
			prev_features = input
		return input

Архитектура была взята из оригинальной статьи (после кажой свертки стоит BatchNorm и ReLu):
- 7 × 7 conv, stride 2
- 3 × 3 max pool, stride 2
- dense layer x 6
- 1 × 1 conv
- 2 × 2 average pool, stride 2
- dense layer x 12
- 1 × 1 conv
- 2 × 2 average pool, stride 2
- dense layer x 24
- 1 × 1 conv
- 2 × 2 average pool, stride 2
- dense layer x 16
- 1 × 1 conv
- 7 × 7 global average pool
- fully-connected, softmax

In [128]:
class Model(nn.Module):
    def __init__(self, num_input_features : int, num_classes : int,
                 init_conv_num_features=24, k=4, bn_size=16, compression=0.5) -> None:
        super().__init__()

        self.blocks = nn.Sequential(OrderedDict([
          ('conv0', nn.Conv2d(num_input_features, init_conv_num_features, 
                              kernel_size=(7, 7), stride=2)),
          ('norm0', nn.BatchNorm2d(init_conv_num_features)),
          ('relu0', nn.ReLU()),
          ('maxpool', nn.MaxPool2d(kernel_size=(3, 3), stride=2))
		]))

        dense_sizes = [6, 12, 24, 16]
        dense_input_features_size = init_conv_num_features
        block_num = 1
        for dense_block_size in dense_sizes:
            self.blocks.add_module(f"dense{block_num}", 
                DenseBlock(num_input_features=dense_input_features_size, 
                            num_layers=dense_block_size, k=k, bn_size=bn_size))
            
            conv_input_feature_size = k * dense_block_size
            conv_output_feature_size = int(conv_input_feature_size * compression)
            self.blocks.add_module(f"compress{block_num}",
				nn.Conv2d(in_channels=conv_input_feature_size,
              			  out_channels=conv_output_feature_size,
                          kernel_size=(1, 1)))
            
            self.blocks.add_module(f"norm{block_num}", nn.BatchNorm2d(conv_output_feature_size))
            self.blocks.add_module(f"relu{block_num}", nn.ReLU())
            # last pooling is 7x7
            if block_num != len(dense_sizes):
                self.blocks.add_module(f"avgpool{block_num}", nn.AvgPool2d(kernel_size=(2, 2), stride=2))
            dense_input_features_size = conv_output_feature_size
            block_num += 1			

        # Classification
        self.blocks.add_module('globavgpool', nn.AvgPool2d(kernel_size=(7, 7)))
        self.classifier = nn.Linear(dense_input_features_size, num_classes)

    def forward(self, x):
        output = self.blocks(x)
        # We don't want to flatten batch
        return self.classifier(torch.flatten(output, start_dim=1))

In [129]:
# если вы написали модель правильно
# эта ячейка должна выполниться
num_input_features = 3
model = Model(num_input_features, num_classes=3)

In [130]:
model

Model(
  (blocks): Sequential(
    (conv0): Conv2d(3, 24, kernel_size=(7, 7), stride=(2, 2))
    (norm0): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU()
    (maxpool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
    (dense1): DenseBlock(
      (dense_blocks): Sequential(
        (denseblock0): DenseLayer(
          (bottle_neck): Sequential(
            (norm1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU()
            (conv1): Conv2d(24, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (conv): Sequential(
            (norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU()
            (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
        )
        (denseblock1): DenseLayer(
          (bottle_neck): Sequ

In [133]:
# Если сделать картинк услишком большой, то она не свернется в 32x1x1 в конце
sample_tensor = torch.randn(1, num_input_features, 320, 320)
model(sample_tensor)

out torch.Size([1, 32, 1, 1])


tensor([[-0.2350,  0.1957,  0.1086]], grad_fn=<AddmmBackward0>)

### Как проводить эксперименты

"Neural net training is a leaky abstraction" - Andrej Karpathy

Знания теории, архитектур, оптимизаторов порой недостаточно для получения хорошей модели - значит, пришла пора подбора гиперпараметров.  
В таких случаях может помочь не *model-centric*, а *data-centric* подход: переразметить данные, поменять аугментации, докинуть новые.

**Но во всех этих случаях правильно организовать эксперименты**



**Перед началом:**
Убедитесь, что у вас есть хороший и адекватный бейзлайн
1. Сначала вместо самописных моделей берите архитектуры из известных репозиториев (torchvision, timm, mmdetection, huggingface etc)
2. Эти архитектуры должны быть стандартными для вашей задачи. То есть, для задач компьютерного зрения (классификации, детекции, сегментации) - ResNet, для обработки языков - трансформер.
3. Не придумывайте сложные пайплайны обучения - Adam + LR без расписания, предобработка входа - такая же как у предобученной модели
4. Первые пробные запуски делайте на подвыборках, тестовых датасетах


**Снизьте число факторов влияния**:
1. Баги могут быть в разных частях: в модели, обучении, загрузке данных, проверке качества. Сначала избавьтесь от эффекта случайности и зафиксируйте seed и попробуйте поставить determenistic поведение
2. Визуализируйте *все*: метрики, лоссы, градиенты, примеры работы модели, работу аугментаций
3. Пишите unit-тесты. Даже небольшие!
4. Сохраняйте чекпоинты. Не только best и last. Полезно брать чекпоинты каждые несколько итераций
5. При проведении экспериментов вносите **только одно изменение за раз**.


Более полные и точные рецепты можете прочитать [здесь](https://github.com/puhsu/dl-hse/blob/main/week01-intro/lecture-best-practices.pdf)

## Обучение и подбор гиперпараметров



> **Гиперпараметры** отличаются от **параметров** следующим:
> * Они не могут обучаться с помощью градиентного спуска: например, выбор оптимизатора, learning rate, аугментаций, сам подбор архитектуры и пайплайна можно считать за гиперпараметры. Иначе говоря, все, что мы не можем включить в нашу end-to-end модель, чтобы обучать это через функцию потерь, является гиперпараметром
> * Часто гиперпараметры подбирают на валидационной выборке (точнее, если их подбор уж очень важен, для них создают специальную выборку, которая называется *dev выборка*): например, weight decay
> * Гиперпараметры бывают дискретными и без отношения порядка: например, выбор расписания для lr, а также выбор момента шага расписания - можно делать шаг на каждом шаге оптимизатора, можно на каждой эпохе



**Вопрос**: почему weight decay лучше подбирать на валидационной выборке? Можно ли его подбирать на обучающей? Если можно, то как?

Чтобы не писать собственный train loop, мы будем использовать **Pytorch Lightning**.   

Это не самый лучший фреймворк для обучения - в нем множество багов, которые особенно любят проявлять себя в сложных моделях, обучаемых в low-precision с параллелизмом.  

Но большая часть популярных фреймворков организована именно так - train loop скрыт от глаз пользователя. Поэтому полезно посмотреть это на таком простом примере, как Pytorch Lightning

In [ ]:
! pip install pytorch_lightning >> None

In [ ]:
import pytorch_lightning as pl

class ConvModelPL(pl.LightningModule):
  def __init__(self, model, lr, weight_decay):
    super().__init__()
    self.model = model
    self.lr = lr
    self.weight_decay = weight_decay

  def training_step(self, batch, batch_idx):
    # training_step определяет шаг в train loop
    # forward модели и подсчет лосса
    x, y = batch
    # <your code here>
    # по умолчанию логгируем в TensorBoard
    self.log("train_loss", loss)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    # соответсвенно, здесь выполняется шаг валидации
    # тоже нужно сделать forward модели и подсчитать лосс
    # но кроме этого - вычислить метрику
    # <your code here>
    self.log("val_loss", loss)
    return metric

  def validation_epoch_end(self, validation_step_outputs):
    # этот шаг выполняется в конце эпохи
    # здесь мы усредним накопленную метрику
    # и передадим ее в логгер
    total_metric = torch.stack(validation_step_outputs).mean()
    self.log("val_epoch_acc", acc_epoch)


  def configure_optimizers(self):
      # здесь мы настраиваем оптимизатор
      # вы можете сделать более сложную конфигурацию
      optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay,)
      return optimizer

In [ ]:
model = ConvNet()
model_pl = ConvModelPL(model, lr=1e-4, weight_decay=1e-6)

Дальше создадим датасеты и даталоадеры.
Опять же, вам нужно написать более точную конфигурацию: подобрать аугментации для baseline, batch_size, параметры даталоадера

In [ ]:
import torchvision
from torchvision import transforms

batch_size = 32
workers = 1

# вспомните, что вы можете использовать не только аугментации из torchvision
# но и из albumentations и, если уж совсем хотите заморочиться, nvidia dali
# прочитайте вот эту статью, возможно, аугментации из нее могут вам помочь
# https://openaccess.thecvf.com/content_CVPR_2019/papers/He_Bag_of_Tricks_for_Image_Classification_with_Convolutional_Neural_Networks_CVPR_2019_paper.pdf
transform = transforms.Compose(
    # <your code here>
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# есть несколько способов ускорения даталоадера

# главный из них - ставить pin_memory, когда вы работаете с gpu
# дело в том, что программы на host'е работает с логической памятью, которая называется paged memory,
# она связана с физической с помощью таблицы - page table
# когда физической памяти не хватает, страницы из page memory выгружаются (page out) на другие носители (например, на ssd)
# получается, paged memory нестабильна и может быть разбросана по разным физическим устройствам
# чтобы скопировать данные на device, сначала данные из paged memory копируются в page-locked memory,
# и только затем на device
# можно избежать такого: сразу выделять память в page-locked memory
# именно это и делает аргумент pin_memory=True
# https://developer.nvidia.com/blog/how-optimize-data-transfers-cuda-cc/

# также если у вашего трейнлупа нет точек синхронизации (напимер, print, logging, перемещение на cpu)
# то можно ставить data = data.to('cuda:0', non_blocking=True) при отправлении данных
# https://discuss.pytorch.org/t/should-we-set-non-blocking-to-true/38234/3

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)

**Вопрос:** на что влияет аргумент num_workers в DataLoader? Каким его можно ставить?

In [ ]:
# а теперь можно запускать обучение и смотреть метрики и графики
# просмотр графиков вы должны вставить сами

# чтобы запустить на маке
# напишите device='mps'
device = 'cuda'

# здесь только пробный запуск
# очевидно, вы должны изменить параметры limit_train_batches и max_epochs
# когда будете делать более сложные эксперименты
trainer = pl.Trainer(limit_train_batches=100, max_epochs=20)
trainer.fit(model_pl, train_loader, test_loader)

Попробуйте получить хороший бейзлайн по рекомендациям выше.  
После обучения модели подберите гиперпараметры и, если вы этого захотите, немного измените архитектуру модели так, чтобы добиться более высокой метрики.

In [ ]:
# что вы можете подбирать: оптимизатор, scheduler, learning rate, аугментации,
# weight decay, тип инициализации
# <your code here>

## Transfer Learning и Fine-Tune

Для многих прикладных задач не существует больших датасетов с хорошей разметкой. Поэтому распространенным приемом является тренировка на похожем, но большом датасете и доучивание сети на целевом.

Такой прием называют **Transfer Learning** или **Fine-tuning**.

В сверточных сетях для классификации выделяют две части:
1. Тело сети (backbone, feature extractor) - это набор сверток и пулингов (convolutions and poolings)
2. Голову (head) - это MLP (набор полносвязных слоев) после которых делается softmax и получаются вероятности разных классов.

Вычислительно простым вариантом finetuning является переучивание головы сети. Также можно фиксировать какие-то первый слои

In [ ]:
from torchvision import models

model = models.resnet18(pretrained=True)

# кроме torchvision очень известен репозиторий pytorch-image-models
# !pip install timm >> None
# import timm
# model = timm.create_model('resnet18', pretrained=True)

In [ ]:
# заморозим слои
for param in model.parameters():
  param.requires_grad = False

In [ ]:
# 10 - число наших классов
model.fc = nn.Linear(in_features=512, out_features=10, bias=True)
# теперь requires_grad=True только у model.fc

**Вопрос:** почему нужно использовать lr warmup для fine-tune предобученной модели?

In [ ]:
# осталось лишь заметить, что пайплайн обучения уже написан - он хранится в model_pl
# вам осталось его только запустить
# проведите несколько экспериментов:
# 1. Дообучите только голову
# 2. Дообучите всю модель
# 3. Поменяйте пайплайн аугментаций с вашего на тот, что использовался для предобученной модели
# 4. Откусите голову и обучите SVM на данных, полученных из feature extractor'a. Попробуйте с аугментациями и без них.
# Такой подход сработает, ведь feature extractor можно рассматривать как функцию, которая отображает данные из одного пространства в другое,
# где эти данные линейно разделимы
# сравните результаты между полной сетью, сетью с дообучением головы и сетью с SVM. Где результаты лучше и почему?

**Вопросы:**  
1. Какая разница по качеству между обучением всей модели и только головы? Как вы думаете, какие преимущества у каждого из этих подходов?
2. Какие зависимости вы обнаружили между различными значениями гиперпараметров и процессом обучения модели?
3. Прочитайте раздел Loss Functions [отсюда](https://cs231n.github.io/neural-networks-2/) (можете и другие разделы). Как вы думаете, почему нельзя обучать классификатор на MSE?